# LangChain Agent Conversation and Memory

Inspiration taken from https://www.youtube.com/watch?v=X550Zbz_ROE

This is important to consider because amount of tokens is not limitless meaning that we need to be wise about the way we store context during an conversation with the LLM

In [100]:
import os
from langchain.prompts  import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

In [ ]:
# Make sure you have setup your environmental variables beforehand
os.environ["OPENAI_API_KEY"]
os.environ["OPENAI_ORGANIZATION"]

Initiate model as usual

In [102]:
llm_openai = OpenAI(model_name="gpt-4-0613", temperature=0, max_tokens=512)

/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/langchain/llms/openai.py:785: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### Look at first ConversationBufferMemory for simpler tasks

Instiate memory where log will be stored. This will be track what user & AI exchange, and stack that into prompt

In [103]:
ai_memory = ConversationBufferMemory()

Start conversational chain. This like LLMChain, can also include a prompt (let's test that in the Playground notebooks)

In [104]:
conversation_chain = ConversationChain(
    llm=llm_openai,
    memory=ai_memory,
    verbose=True
)

In [105]:
conversation_chain.predict(input="Hi, I'm El Brian")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm El Brian
AI:



> Finished chain.


"Hello El Brian! It's a pleasure to meet you. I'm an AI developed to assist with information and answer any questions you might have. How can I assist you today?"

Notice how it keeps track of the conversation even as new queries are requested

In [106]:
conversation_chain.predict(input="how is your afternoon so far?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm El Brian
AI: Hello El Brian! It's a pleasure to meet you. I'm an AI developed to assist with information and answer any questions you might have. How can I assist you today?
Human: how is your afternoon so far?
AI:

> Finished chain.


"As an artificial intelligence, I don't experience time or have personal feelings. But thank you for asking! How can I assist you further?"

Notice below the memory object where AI/User exchange is logged

In [107]:
print(ai_memory.chat_memory.messages)

[HumanMessage(content="Hi, I'm El Brian", additional_kwargs={}, example=False), AIMessage(content="Hello El Brian! It's a pleasure to meet you. I'm an AI developed to assist with information and answer any questions you might have. How can I assist you today?", additional_kwargs={}, example=False), HumanMessage(content='how is your afternoon so far?', additional_kwargs={}, example=False), AIMessage(content="As an artificial intelligence, I don't experience time or have personal feelings. But thank you for asking! How can I assist you further?", additional_kwargs={}, example=False)]


This is also stored in the actual Conversation Chain object

In [108]:
print(conversation_chain.memory.buffer)

Human: Hi, I'm El Brian
AI: Hello El Brian! It's a pleasure to meet you. I'm an AI developed to assist with information and answer any questions you might have. How can I assist you today?
Human: how is your afternoon so far?
AI: As an artificial intelligence, I don't experience time or have personal feelings. But thank you for asking! How can I assist you further?


The above type of memory (ConversationBufferMemory) works well where limited exchanges are expected (~5). Let's look at longer memory types below

### Let's explore ConversationSummaryMemory for some more exchange intensives scenarios

In [109]:
from langchain.memory import ConversationSummaryMemory

Notice how this initiation of conversation memory does require the llm input (unlike ConversationMemory above)

<u>Why the difference?</u> Because it uses an additional LLM to summarise the log, besides the LLM use for the  (Test multi-LLM usage in Playground)

In [110]:
ai_summary_memory = ConversationSummaryMemory(llm=llm_openai)

In [111]:
conversation_chain = ConversationChain(
    llm=llm_openai,
    memory=ai_summary_memory,
    verbose=True
)

In [112]:
conversation_chain.predict(input="Hi, I'm El Brian")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm El Brian
AI:

> Finished chain.


"Hello El Brian! It's nice to meet you. I'm an AI developed to assist with information and answer questions to the best of my ability. How can I assist you today?"

Notice the key summary difference now

In [113]:
conversation_chain.predict(input="How are you doing this evening?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
El Brian introduces himself to the AI. The AI responds by introducing itself as an assistant developed to provide information and answer questions. It then asks El Brian how it can assist him.
Human: How are you doing this evening?
AI:

> Finished chain.


"As an artificial intelligence, I don't have feelings or personal experiences, but I'm here and ready to assist you. How can I help you this evening?"

They AI summarizes the prior conversation, rather than logging the entire conversation

In [114]:
print(conversation_chain.memory.buffer)

El Brian introduces himself to the AI, which introduces itself as an assistant developed to provide information and answer questions. When El Brian asks how the AI is doing, it explains that as an AI, it doesn't have feelings or personal experiences, but it's ready to assist. It then asks El Brian how it can help him this evening.


Notice however in the example that at first we are using more tokens. Let's add more to the log to keep seeing how summarization works

In [115]:
conversation_chain.predict(input="Can you help me with identifying the best country to visit in the summer?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
El Brian introduces himself to the AI, which introduces itself as an assistant developed to provide information and answer questions. When El Brian asks how the AI is doing, it explains that as an AI, it doesn't have feelings or personal experiences, but it's ready to assist. It then asks El Brian how it can help him this evening.
Human: Can you help me with identifying the best country to visit in the summer?
AI:

> Finished chain.


'Absolutely, I\'d be happy to help with that. However, the "best" country to visit in the summer can depend on a variety of factors such as your personal interests, budget, and the type of experience you\'re looking for. \n\nFor example, if you\'re interested in history and culture, Italy or Greece could be great options. They offer numerous historical sites, delicious cuisine, and warm Mediterranean weather. \n\nIf you\'re a nature lover, you might consider visiting Canada or Norway. Both countries offer stunning natural landscapes, including mountains, forests, and lakes. The summer months are also a great time to visit these countries as the weather is generally pleasant and the days are long.\n\nFor beach lovers, countries like Australia, Indonesia (Bali), or the Maldives might be appealing. They offer beautiful beaches and a variety of water activities.\n\nIf you\'re looking for a unique cultural experience, Japan could be an interesting choice. Summer is the season for many tradi

In [116]:
print(conversation_chain.memory.buffer)

El Brian introduces himself to the AI, which introduces itself as an assistant developed to provide information and answer questions. When El Brian asks how the AI is doing, it explains that as an AI, it doesn't have feelings or personal experiences, but it's ready to assist. It then asks El Brian how it can help him this evening. El Brian asks the AI to help him identify the best country to visit in the summer. The AI explains that the "best" country can depend on various factors like personal interests, budget, and the type of experience one is looking for. It suggests Italy or Greece for history and culture enthusiasts, Canada or Norway for nature lovers, Australia, Indonesia, or the Maldives for beach lovers, and Japan for a unique cultural experience. The AI then asks El Brian for more details to provide a more tailored recommendation.


The above is a nice option to store memory in summary so that the 'important' context is maximized in memory (tokens). Let's look at an alternate version of the memory where we store memory (tokens) based on a pre-defined token window

### Let's explore ConversationBufferWindowMemory to test this pre-defined token-windowed approach

In [117]:
from langchain.memory import ConversationBufferWindowMemory

The conversation buffer window memory function sets the last number of interactions we want to keep. We are using 2 as an example below

In [118]:
window_memory = ConversationBufferWindowMemory(k=2)

We pass it to the conversation chain memory argument (as before)

In [119]:
conversation_chain = ConversationChain(
    llm=llm_openai,
    memory=window_memory,
    verbose=True
)

In [120]:
conversation_chain.predict(input="I'm looking for travel support")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I'm looking for travel support
AI:

> Finished chain.


"Sure, I'd be happy to help you with that. When it comes to travel support, there are several areas I can assist with. \n\n1. Destination Research: I can provide information about various destinations, including local customs, must-see attractions, best times to visit, and more.\n\n2. Flight and Accommodation: I can help you find the best deals on flights and accommodations based on your preferences and budget.\n\n3. Itinerary Planning: I can suggest an itinerary based on your interests and the duration of your stay.\n\n4. Travel Tips: I can provide tips on packing, local transportation, safety, and more.\n\n5. Local Cuisine: I can suggest local dishes to try and the best places to eat.\n\n6. Emergency Information: I can provide information about local hospitals, embassies, and other important contacts.\n\nPlease let me know which area you need help with, or if there's something else you're looking for."

In [121]:
conversation_chain.predict(input="I would like suggestions for the best places to travel in the US with a low budget")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm looking for travel support
AI: Sure, I'd be happy to help you with that. When it comes to travel support, there are several areas I can assist with. 

1. Destination Research: I can provide information about various destinations, including local customs, must-see attractions, best times to visit, and more.

2. Flight and Accommodation: I can help you find the best deals on flights and accommodations based on your preferences and budget.

3. Itinerary Planning: I can suggest an itinerary based on your interests and the duration of your stay.

4. Travel Tips: I can provide tips on packing, local transportation, safety, and more.

5. Local Cuisine: I can sugges

"Absolutely, I can suggest a few budget-friendly travel destinations in the US:\n\n1. **Washington, D.C.**: The capital city has numerous free museums and historical sites, including the Smithsonian museums, the National Mall, and the Lincoln Memorial. \n\n2. **Yellowstone National Park, Wyoming**: This is a great option if you enjoy outdoor activities. The park's entrance fee is relatively low, and camping can be a cost-effective accommodation option.\n\n3. **Austin, Texas**: Known for its vibrant music scene, Austin offers affordable food and entertainment options. You can also enjoy outdoor activities at Lady Bird Lake.\n\n4. **New Orleans, Louisiana**: The city is rich in culture, music, and food. You can find budget-friendly accommodations and enjoy free street performances in the French Quarter.\n\n5. **Portland, Oregon**: Known for its food trucks, microbreweries, and beautiful parks, Portland can be a budget-friendly option, especially if you're a foodie or nature lover.\n\n6. 

Notice that as before (non-summary chain) we have our two past outputs

In [122]:
print(conversation_chain.memory.buffer)

Human: I'm looking for travel support
AI: Sure, I'd be happy to help you with that. When it comes to travel support, there are several areas I can assist with. 

1. Destination Research: I can provide information about various destinations, including local customs, must-see attractions, best times to visit, and more.

2. Flight and Accommodation: I can help you find the best deals on flights and accommodations based on your preferences and budget.

3. Itinerary Planning: I can suggest an itinerary based on your interests and the duration of your stay.

4. Travel Tips: I can provide tips on packing, local transportation, safety, and more.

5. Local Cuisine: I can suggest local dishes to try and the best places to eat.

6. Emergency Information: I can provide information about local hospitals, embassies, and other important contacts.

Please let me know which area you need help with, or if there's something else you're looking for.
Human: I would like suggestions for the best places to t

In [123]:
conversation_chain.predict(input="Where should I stay in Austin Texas?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I'm looking for travel support
AI: Sure, I'd be happy to help you with that. When it comes to travel support, there are several areas I can assist with. 

1. Destination Research: I can provide information about various destinations, including local customs, must-see attractions, best times to visit, and more.

2. Flight and Accommodation: I can help you find the best deals on flights and accommodations based on your preferences and budget.

3. Itinerary Planning: I can suggest an itinerary based on your interests and the duration of your stay.

4. Travel Tips: I can provide tips on packing, local transportation, safety, and more.

5. Local Cuisine: I can sugges

"Austin, Texas, has a variety of neighborhoods that cater to different interests and budgets. Here are a few suggestions:\n\n1. **Downtown Austin**: This is the heart of the city with easy access to restaurants, bars, and attractions like the Texas State Capitol and Lady Bird Lake. It might be a bit pricier, but the convenience could be worth it.\n\n2. **East Austin**: This is a trendy, up-and-coming area with a mix of bungalows and modern condos. It's known for its vibrant nightlife, art scene, and food trucks.\n\n3. **South Congress (SoCo)**: This area is known for its shopping, dining, and entertainment. It's a bit more laid-back than downtown but still has plenty of attractions.\n\n4. **Hyde Park**: If you're looking for a quieter, more residential area, Hyde Park might be a good fit. It's a historic neighborhood with easy access to the University of Texas at Austin.\n\n5. **Rainey Street District**: This area is known for its historic homes turned into bars and restaurants. It's a

Notice now that it has lost the first interaction. <u>This is what window memory buffer does</u>, it allows the user to manually set how many windows of memory to keep, therefore reducing the amount of tokens required at the end

In [124]:
print(conversation_chain.memory.buffer)

Human: I would like suggestions for the best places to travel in the US with a low budget
AI: Absolutely, I can suggest a few budget-friendly travel destinations in the US:

1. **Washington, D.C.**: The capital city has numerous free museums and historical sites, including the Smithsonian museums, the National Mall, and the Lincoln Memorial. 

2. **Yellowstone National Park, Wyoming**: This is a great option if you enjoy outdoor activities. The park's entrance fee is relatively low, and camping can be a cost-effective accommodation option.

3. **Austin, Texas**: Known for its vibrant music scene, Austin offers affordable food and entertainment options. You can also enjoy outdoor activities at Lady Bird Lake.

4. **New Orleans, Louisiana**: The city is rich in culture, music, and food. You can find budget-friendly accommodations and enjoy free street performances in the French Quarter.

5. **Portland, Oregon**: Known for its food trucks, microbreweries, and beautiful parks, Portland can

Let's look at now an alternative version, which is a combination of memory buffer of sorts, which mixes a max token limit over re-created summaries

### Let's explore ConversationSummaryBufferMemory which applies a maximum token threshold over summaries

In [125]:
from langchain.memory import ConversationSummaryBufferMemory

Let's load the memory class, and as before with ConversationSummaryMemory(), notice that it needs an llm to also summarize context. Rather than a window also notice that we are using max tokens to threshold memory usage

In [126]:
memory = ConversationSummaryBufferMemory(llm=llm_openai,
                                          max_token_limit=50)

In [127]:
conversation_chain = ConversationChain(
    llm=llm_openai,
    memory=memory,
    verbose=True
)

In [128]:
conversation_chain.predict(input="Hi, this is El Brain")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, this is El Brain
AI:

> Finished chain.


"Hello El Brain! It's a pleasure to meet you. I'm an AI developed to assist with a variety of tasks and answer any questions you might have to the best of my ability. How can I assist you today?"

In [129]:
conversation_chain.predict(input="I need help with travel arrengement within the US")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human introduces themselves as El Brain.
AI: Hello El Brain! It's a pleasure to meet you. I'm an AI developed to assist with a variety of tasks and answer any questions you might have to the best of my ability. How can I assist you today?
Human: I need help with travel arrengement within the US
AI:

> Finished chain.


"Of course, El Brain! I'd be happy to assist you with your travel arrangements. Could you please provide me with more details? For instance, your departure and destination cities, preferred dates of travel, whether you need assistance with flight bookings, hotel reservations, car rentals, or all of these? The more specific you are, the better I can assist you."

In [130]:
conversation_chain.predict(input="I want to travel over the summer within low budget")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: El Brain introduces themselves to the AI, which responds by offering its assistance with a variety of tasks. El Brain asks for help with travel arrangements within the US, and the AI asks for more specific details such as departure and destination cities, preferred dates, and whether assistance is needed with flight bookings, hotel reservations, or car rentals.
Human: I want to travel over the summer within low budget
AI:

> Finished chain.


"That sounds like a great plan! To help you better, I'll need a bit more information. Could you please specify:\n\n1. Your departure city and destination city.\n2. The exact dates or range of dates you're considering for your travel.\n3. Your budget range.\n4. Your preferences for accommodation (e.g., hotel, hostel, Airbnb).\n5. If you need a rental car or if you're planning to use public transportation.\n6. Any specific activities or attractions you're interested in, which could affect the location of your accommodation.\n\nWith these details, I can provide a more tailored and budget-friendly travel plan for you."

In [131]:
conversation_chain.predict(input="I'm looking into cities in the mid west")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: El Brain introduces themselves to the AI, which responds by offering its assistance with a variety of tasks. El Brain asks for help with travel arrangements within the US, and the AI asks for more specific details such as departure and destination cities, preferred dates, and whether assistance is needed with flight bookings, hotel reservations, or car rentals. El Brain then specifies they want to travel over the summer on a low budget. The AI asks for further information including exact dates, budget range, accommodation preferences, transportation needs, and any specific activities or attractions of interest to provide a more tailored and budget-friendly trav

"That's great! The Midwest has a lot to offer in terms of culture, history, and natural beauty. Some popular cities to consider are Chicago, Illinois; Minneapolis, Minnesota; St. Louis, Missouri; and Indianapolis, Indiana. Each of these cities has its own unique attractions. For example, Chicago is known for its architecture, museums, and food scene; Minneapolis for its lakes and parks; St. Louis for the Gateway Arch and its breweries; and Indianapolis for the Indianapolis Motor Speedway and its vibrant arts scene.\n\nIf you could provide more details about what you're interested in, such as outdoor activities, historical sites, food and drink, arts and culture, or sports, I could help you narrow down your options and suggest specific attractions. Also, knowing your exact travel dates would help me check for any special events or festivals happening in these cities during your visit."

Notice two things. The return object contain within the conversation chain is "moving_summary_buffer" rather than "buffer" AND the summary window is constrained to the 50 token limit

In [132]:
print(conversation_chain.memory.moving_summary_buffer)

El Brain introduces themselves to the AI, which responds by offering its assistance with a variety of tasks. El Brain asks for help with travel arrangements within the US, and the AI asks for more specific details such as departure and destination cities, preferred dates, and whether assistance is needed with flight bookings, hotel reservations, or car rentals. El Brain then specifies they want to travel over the summer on a low budget. The AI asks for further information including exact dates, budget range, accommodation preferences, transportation needs, and any specific activities or attractions of interest to provide a more tailored and budget-friendly travel plan. El Brain expresses interest in visiting cities in the Midwest, and the AI suggests popular cities like Chicago, Minneapolis, St. Louis, and Indianapolis, each with its own unique attractions. The AI then asks for more details about El Brain's interests and exact travel dates to suggest specific attractions and check for 

This is a great way to summarize memory with token limit. Let's look at alternative more algorithmically way to handle memory

### Let's take a look at Conversation Knowledge Graph Memory (ConversationKGMemory)

In [133]:
from langchain.memory import ConversationKGMemory

This type of memory enforces use of specific information only presented in a section of the prompt (<u>history</u>) and provides a PromptTemplate to do the AI to do so.

The intent is to <u>reduce hallucination and focus on specific context</u>

In [134]:
template = """The following is a conversation between and AI and a Human. The AI is helpful and provides specific details from its context.
If the AI doesn't know an answer to the question, it responds with I don't know. 
The AI will only uses information contained in the Relevant information section and does not hallucinate.


Relevant information

{history}

Conversation: 
Human: {input}
AI:"""

In [135]:
prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)

In addition we also instantiate the memory class with an llm (doesn't have to be gpt3.5 here, or before) just like with summary memory chains

In [136]:
memory = ConversationKGMemory(llm=llm_openai)

Putting all together into the conversation chain with the prompt we get:

In [137]:
conversation_kgm = ConversationChain(
    llm=llm_openai,
    memory=memory,
    prompt=prompt,
    verbose=True
)

In [138]:
conversation_kgm.predict(input="Hi I'm El Brain")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between and AI and a Human. The AI is helpful and provides specific details from its context.
If the AI doesn't know an answer to the question, it responds with I don't know. 
The AI will only uses information contained in the Relevant information section and does not hallucinate.


Relevant information



Conversation: 
Human: Hi I'm El Brain
AI:

> Finished chain.


'Hello El Brain, how can I assist you today?'

In [139]:
conversation_kgm.predict(input="I need help with travel arrengements in the US")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between and AI and a Human. The AI is helpful and provides specific details from its context.
If the AI doesn't know an answer to the question, it responds with I don't know. 
The AI will only uses information contained in the Relevant information section and does not hallucinate.


Relevant information



Conversation: 
Human: I need help with travel arrengements in the US
AI:

> Finished chain.


"I'm sorry, but I don't have any specific information about travel arrangements in the US."

The AI can't provide an answer to the request given that there is no associated context under the "Relevant information" section

In [140]:
conversation_kgm.predict(input="I'm not sure how I can book travel")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between and AI and a Human. The AI is helpful and provides specific details from its context.
If the AI doesn't know an answer to the question, it responds with I don't know. 
The AI will only uses information contained in the Relevant information section and does not hallucinate.


Relevant information



Conversation: 
Human: I'm not sure how I can book travel
AI:

> Finished chain.


"I'm sorry, but I don't have any information on how to book travel."

Let's try however providing context and relevant infornation it can work on to help identify an answer from relationships in the text

In [141]:
conversation_kgm.predict(input="My computer is broken and not turning on")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between and AI and a Human. The AI is helpful and provides specific details from its context.
If the AI doesn't know an answer to the question, it responds with I don't know. 
The AI will only uses information contained in the Relevant information section and does not hallucinate.


Relevant information



Conversation: 
Human: My computer is broken and not turning on
AI:

> Finished chain.


"I'm sorry to hear that. However, I need more information to help you troubleshoot the problem. Can you tell me what happens when you try to turn it on?"

In [142]:
conversation_kgm.predict(input="The screen is still black, it doesn't do the macOS sound like it usually does")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between and AI and a Human. The AI is helpful and provides specific details from its context.
If the AI doesn't know an answer to the question, it responds with I don't know. 
The AI will only uses information contained in the Relevant information section and does not hallucinate.


Relevant information



Conversation: 
Human: The screen is still black, it doesn't do the macOS sound like it usually does
AI:

> Finished chain.


"It seems like your Mac is not starting up properly. Unfortunately, I don't have enough information to provide a specific solution. It could be a hardware issue or a software problem. You might want to try some general troubleshooting steps like checking the power source, trying a different display, or booting in safe mode."

The Knowledge Graph Memory (KGM) chain used provided text to identify a suitable answer by building a knolwedge graph. Let's look at the memory collected and the relationship between the entities as the KGM LLM tried to identify relationship

In [143]:
conversation_kgm.memory.kg.get_triples()

[('El Brain', 'Human', 'is'),
 ('Human', 'travel arrangements in the US', 'needs help with'),
 ('My computer', 'broken', 'is'),
 ('My computer', 'turning on', 'is not'),
 ("Human's computer", 'black screen', 'has'),
 ("Human's computer", 'macOS startup sound', "doesn't do")]

It would be great to <u> identify the history stored object </u> it used for context but it was not initially apparent. We can however look at the memory exchange

In [99]:
conversation_kgm.memory.chat_memory.messages

[HumanMessage(content="Hi I'm El Brain", additional_kwargs={}, example=False),
 AIMessage(content='Hello El Brain, how can I assist you today?', additional_kwargs={}, example=False),
 HumanMessage(content='I need help with travel arrengements in the US', additional_kwargs={}, example=False),
 AIMessage(content="I'm sorry, but I don't have any specific information about travel arrangements in the US.", additional_kwargs={}, example=False),
 HumanMessage(content="I'm not sure how I can book travel", additional_kwargs={}, example=False),
 AIMessage(content="I'm sorry, but I don't have the specific information on how to book travel.", additional_kwargs={}, example=False),
 HumanMessage(content='My computer is broken and not turning on', additional_kwargs={}, example=False),
 AIMessage(content="I'm sorry to hear that. However, I need more information to help you troubleshoot the problem. Can you tell me what happens when you try to turn it on?", additional_kwargs={}, example=False),
 HumanM

### Lastly let's take a look at the Entity Memory

In [144]:
from langchain.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

Notice the standard template and its use of the <u>entities argument</u>

In [146]:
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations and de

Let's start the converesation chain as usual with <u>ConversationChain</u>. Notice how we pass the template above (so an entities section is expected by the template prompt) and also an llm to support entity memory use

In [153]:
conversation_entity = ConversationChain(
    llm=llm_openai,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm_openai),
    verbose=True
)

In [154]:
conversation_entity.predict(input="Hi, I'm El Brain. My cat, his name is Gordo is he is hiding")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"Hello El Brain! It's nice to meet you. Cats like Gordo often hide for a variety of reasons. It could be due to fear, stress, or simply because they found a new cozy spot. If Gordo is hiding more than usual, it might be a good idea to check if there are any changes in his environment that could be causing him stress. If he continues to hide and shows signs of distress or illness, it would be best to consult with a vet."

In [155]:
conversation_entity.predict(input="How can I help him? His insurnace number is B0456C7") # Might need to change this to include entity example!!



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"To help Gordo feel more comfortable, you can try the following:\n\n1. **Create a Safe Space:** Make sure Gordo has a safe, quiet space where he can retreat to. This could be a specific room, a cat tree, or even a box. Cats often feel safer in elevated areas or enclosed spaces.\n\n2. **Minimize Stress:** Try to identify any potential sources of stress in Gordo's environment and minimize them if possible. This could include loud noises, unfamiliar people or animals, or changes in his routine.\n\n3. **Play and Interaction:** Spend quality time with Gordo. Play with him using his favorite toys. This can help build trust and make him feel more comfortable.\n\n4. **Familiar Scents:** Cats are very scent-oriented. You can try leaving items with your scent in his favorite hiding places to make him feel more secure.\n\n5. **Gradual Introduction:** If there are new elements in his environment (like a new pet or family member), introduce them gradually and always under positive circumstances.\n\

By providing context to the Conversation Entity Memory Chain it can collect entities through the conversaion (review template) to get a sense of relevant specific subjects/topics that needs to capture. Below is the entity identified here.

In [156]:
conversation_entity.memory.entity_cache

['B0456C7']

In [157]:
conversation_entity.predict(input="Can you help me contact the Vet?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"I'm sorry for any confusion, but as an AI, I don't have the ability to make phone calls or send emails. However, I can guide you on how to do it. \n\nHere are the general steps to contact a vet:\n\n1. **Find the Contact Information:** Look up the contact information for your vet. This could be a phone number or an email address. It's usually available on their website.\n\n2. **Prepare Information:** Before you contact the vet, prepare all the necessary information. This includes Gordo's symptoms, any changes in behavior, and his insurance number (B0456C7).\n\n3. **Make the Call or Send the Email:** If you're calling, wait until you're connected to a representative or leave a voicemail with all the necessary information. If you're emailing, make sure to include all the information in a clear and concise manner.\n\n4. **Follow Their Instructions:** The vet or their representative will likely give you instructions on what to do next. This could include bringing Gordo in for a check-up, m

We can also take a look at all the entities and their context in the conversation

In [163]:
print(conversation_entity.memory.entity_store)

store={'El Brain': 'El Brain is a person who has a cat named Gordo.', 'Gordo': "Gordo is El Brain's cat who has a tendency to hide.", 'B0456C7': "B0456C7 is the insurance number for El Brain's cat, Gordo."}


### Wrap up

In this exercise we've taken a look at the different ways to incorporate memory chains in the interaction with LLMs. These have include simple interactions (<u>ConversationChain</u>) to more sophisticated approaches to optimally utilize token while retaining context in which summaries and maximum token buffer windows area allowed <u>ConversationSummaryBufferMemory</u>

We can develop this further by incorporating PromptTemplates in our Playground noteboooks